<a href="https://colab.research.google.com/github/aguzel/computational_imaging/blob/main/CompLightPercep_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from PIL import Image
import numpy as np_cpu
import matplotlib.pyplot as plt
import matplotlib.colors

import torch
import torch.optim as optim

from torchvision import transforms


%matplotlib inline

In [2]:
def load_image(fn):
    """
    Definition to load an image from a given location as a Numpy array.
    Parameters
    ----------
    fn           : str
                   Filename.
    Returns
    ----------
    image        :  ndarray
                    Image loaded as a Numpy array.
    """
    image = Image.open(fn)
    image = np_cpu.array(image)
    image = torch.from_numpy(image)
    return image

In [3]:
def rgb_2_ycrcb(image):
    """
    Converts an image from RGB colourspace to YCrCb colourspace.
    Parameters
    ----------
    image   : torch.tensor
                Input image. Should be an RGB floating-point image with values in the range [0, 1]
                Should be in NCHW format.
    Returns
    -------
    ycrcb   : torch.tensor
                Image converted to YCrCb colourspace.
    """
    ycrcb = torch.zeros(image.size()).to(image.device)
    ycrcb[:, 0, :, :] = 0.299 * image[:, 0, :, :] + 0.587 * \
        image[:, 1, :, :] + 0.114 * image[:, 2, :, :]
    ycrcb[:, 1, :, :] = 0.5 + 0.713 * (image[:, 0, :, :] - ycrcb[:, 0, :, :])
    ycrcb[:, 2, :, :] = 0.5 + 0.564 * (image[:, 2, :, :] - ycrcb[:, 0, :, :])
    return ycrcb

In [ ]:
# Im = Image.open('/content/image2.png')
# Im

In [6]:
tensor_image_1 = load_image('/content/image2.png')
tensor_image_1 = tensor_image_1/255.0
tensor_image_2 = load_image('/content/image2.png')
tensor_image_2 = tensor_image_2/255.0

In [7]:
nchw_image_1 = torch.swapaxes(tensor_image_1,0,2)
nchw_image_2 = torch.swapaxes(tensor_image_2,0,2)
print(nchw_image_1.unsqueeze(0).shape) # Add batch dimension for NCHW, prints "torch.Size([1, 3, 2, 2])"
print(nchw_image_2.unsqueeze(0).shape) # Add batch dimension for NCHW, prints "torch.Size([1, 3, 2, 2])"
nchw_image_1 = nchw_image_1.unsqueeze(0)
nchw_image_2 = nchw_image_2.unsqueeze(0)

torch.Size([1, 3, 1001, 1001])
torch.Size([1, 3, 1001, 1001])


In [8]:
ycrcb_image_1 = rgb_2_ycrcb(nchw_image_1)
ycrcb_image_2 = rgb_2_ycrcb(nchw_image_2)

In [9]:
def ycrcb_2_rgb(image):
    """
    Converts an image from YCrCb colourspace to RGB colourspace.
    Parameters
    ----------
    image   : torch.tensor
                Input image. Should be a YCrCb floating-point image with values in the range [0, 1]
                Should be in NCHW format.
    Returns
    -------
    rgb     : torch.tensor
                Image converted to RGB colourspace.
    """
    rgb = torch.zeros(image.size(), device=image.device)
    rgb[:, 0, :, :] = image[:, 0, :, :] + 1.403 * (image[:, 1, :, :] - 0.5)
    rgb[:, 1, :, :] = image[:, 0, :, :] - 0.714 * \
        (image[:, 1, :, :] - 0.5) - 0.344 * (image[:, 2, :, :] - 0.5)
    rgb[:, 2, :, :] = image[:, 0, :, :] + 1.773 * (image[:, 2, :, :] - 0.5)
    return rgb

In [12]:
rgb_1 = ycrcb_2_rgb(ycrcb_image_1)
rgb_1 = rgb_1.squeeze(0)
rgb_1 = rgb_1.swapaxes(0,2)
rgb_1.size()


torch.Size([1001, 1001, 3])

In [ ]:
image_np_array_1 = plt.imread('/content/image2.png')
np_arr = rgb_1.cpu().detach().numpy()
plt.imshow(np_arr)

In [21]:
ycrcb_image_2.size()

torch.Size([1, 3, 1001, 1001])

In [42]:
loss = torch.nn.MSELoss()
input = torch.zeros(1,3,1001,1001,requires_grad=True)
target = ycrcb_image_1
output = loss(input, target)
output.backward()

In [44]:
optimizer = optim.Adam(params=,lr=0.001)
input = torch.zeros(1,3,1001,1001,requires_grad=True)
niter = 10
for _ in range(0, niter):
	optimizer.zero_grad()
	loss = loss(input, target)
	loss.backward()
	optimizer.step()

TypeError: ignored